In [6]:
import sys
sys.path.append("../../helios-server/")
import helios.models as models
from helios_auth import models as auth_model
import helios.views as views
from helios.models import CastVote, Voter, Trustee
from helios.workflows.homomorphic import EncryptedVote, Tally, EncryptedAnswer,DLogTable
from helios.crypto import algs
from election_utils import *
from tqdm import tqdm, trange
# import click
import time
import os
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
import pickle

In [7]:
# num_questions = 4
num_choices = 4
choices = [i for i in range(num_choices)]

In [8]:
possible_answers = []
for i in trange(num_choices+1):
    possible_answers.extend([list(xs) for xs in combinations(choices, i)])

100%|██████████| 5/5 [00:00<00:00, 60787.01it/s]


In [ ]:
possible_answers

[[],
 [0],
 [1],
 [2],
 [3],
 [0, 1],
 [0, 2],
 [0, 3],
 [1, 2],
 [1, 3],
 [2, 3],
 [0, 1, 2],
 [0, 1, 3],
 [0, 2, 3],
 [1, 2, 3],
 [0, 1, 2, 3]]

In [20]:
election_keypair = pickle.load(open("election_keypair.p", "rb"))

In [9]:
keys = views.ELGAMAL_PARAMS.generate_keypair()

In [10]:
pickle.dump(keys, open("EGkeypair.p", "wb"))

In [11]:
keys = pickle.load(open("EGkeypair.p", "rb"))

In [22]:
# election_keypair["public_key"]

In [12]:
def fromElectionAndAnswer(choice_indicies, answer, pk, q_max, q_min=0):
    """
    Given an election, a question number, and a list of answers to that question
    in the form of an array of 0-based indexes into the answer array,
    produce an EncryptedAnswer that works.
    """
    
    # initialize choices, individual proofs, randomness and overall proof
    choices = [None for a in range(len(choice_indicies))]
    individual_proofs = [None for a in range(len(choice_indicies))]
    overall_proof = None
    randomness = [None for a in range(len(choice_indicies))]
    
    # possible plaintexts [0, 1]
    plaintexts = EncryptedAnswer.generate_plaintexts(pk)
    
    # keep track of number of options selected.
    num_selected_answers = 0
    
    # homomorphic sum of all
    homomorphic_sum = 0
    randomness_sum = 0

    # min and max for number of answers, useful later

    min_answers = q_min
    max_answers = q_max

    # go through each possible answer and encrypt either a g^0 or a g^1.
    for answer_num in range(len(choice_indicies)):
      plaintext_index = 0
      
      # assuming a list of answers
      if answer_num in answer:
        plaintext_index = 1
        num_selected_answers += 1

      # randomness and encryption
      randomness[answer_num] = algs.random.mpz_lt(pk.q)
      choices[answer_num] = pk.encrypt_with_r(plaintexts[plaintext_index], randomness[answer_num])
      
      # generate proof
      individual_proofs[answer_num] = choices[answer_num].generate_disjunctive_encryption_proof(plaintexts, plaintext_index, 
                                                randomness[answer_num], algs.EG_disjunctive_challenge_generator)
                                                
      # sum things up homomorphically if needed
      if max_answers is not None:
        homomorphic_sum = choices[answer_num] * homomorphic_sum
        randomness_sum = (randomness_sum + randomness[answer_num]) % pk.q

    # prove that the sum is 0 or 1 (can be "blank vote" for this answer)
    # num_selected_answers is 0 or 1, which is the index into the plaintext that is actually encoded
    
    if num_selected_answers < min_answers:
      raise Exception("Need to select at least %s answer(s)" % min_answers)
    
    if max_answers is not None:
      sum_plaintexts = EncryptedAnswer.generate_plaintexts(pk, min=min_answers, max=max_answers)
    
      # need to subtract the min from the offset
      overall_proof = homomorphic_sum.generate_disjunctive_encryption_proof(sum_plaintexts, num_selected_answers - min_answers, randomness_sum, algs.EG_disjunctive_challenge_generator);
    else:
      # approval voting
      overall_proof = None
    
    return EncryptedAnswer(choices, individual_proofs, overall_proof, randomness, answer)

In [13]:
# for num_choices in range(1, 4):
choices = [i for i in range(num_choices)]
print(f"Enumerating possible answers with {num_choices} choices")
possible_answers = []
for i in trange(num_choices+1):
    possible_answers.extend([list(xs) for xs in combinations(choices, i)])

print(f"Generating encyptions")
encrypted_answers = [fromElectionAndAnswer(choices, answer, keys.pk, num_choices) for answer in tqdm(possible_answers)]
# np.save(f"encrypted_answers/all_answers_with_{num_choices}_choices.npy", encypted_answers,  allow_pickle=True)

Enumerating possible answers with 4 choices


100%|██████████| 5/5 [00:00<00:00, 64329.82it/s]


Generating encyptions


100%|██████████| 16/16 [00:03<00:00,  4.21it/s]


In [27]:
list(range(1, 5))

[1, 2, 3, 4]

In [20]:
xs = np.load(f"encrypted_answers/all_answers_with_{num_choices}_choices.npy",  allow_pickle=True)

In [55]:
encrypted_answers[-1].answer

[0, 1, 2]

In [56]:
[encrypted_answers[-1]]

In [57]:
keys.sk.decrypt(encrypted_answers[-1].choices[0]).m

14887492224963187634282421537186040801304008017743492304481737382571933937568724473847106029915040150784031882206090286938661464458896494215273989547889201144857352611058572236578734319505128042602372864570426550855201448111746579871811249114781674309062693442442368697449970648232621880001709535143047913661432883287150003429802392229361583608686643243349727791976247247948618930423866180410558458272606627111270040091203073580238905303994472202930783207472394578498507764703191288249547659899997131166130259700604433891232298182348403175947450284433411265966789131024573629546048637848902243503970966798589660808533

{0: {'answers': range(0, 3)}, 1: {'answers': range(0, 3)}}

In [60]:
np.random.choice(encrypted_answers, 3)

array([<helios.workflows.homomorphic.EncryptedAnswer object at 0x7f03437bce20>,
      dtype=object)

In [14]:
def decrypt_and_prove(tally, sk):
    """
    returns an array of tallies and a corresponding array of decryption proofs.
    """
    dlog_table = DLogTable(base = tally.public_key.g, modulus = tally.public_key.p)
    dlog_table.precompute(tally.num_tallied)

        
    # for all choices of all questions (double list comprehension)
    decrypted_tally = []
    decryption_proof = []

    for question_num in range(len(tally.questions)):
        question = tally.questions[question_num]
        answers = question['answers']
        question_tally = []
        question_proof = []

        for answer_num in range(len(answers)):
            # do decryption and proof of it
            plaintext, proof = sk.prove_decryption(tally.tally[question_num][answer_num])

            # look up appropriate discrete log
            question_tally.append(dlog_table.lookup(plaintext))
            question_proof.append(proof)
        
        decrypted_tally.append(question_tally)
        decryption_proof.append(question_proof)

    return decrypted_tally, decryption_proof

In [15]:
tally.num_tallied

NameError: name 'tally' is not defined

In [16]:
encrypted_answers[0].verify(keys.pk)

bad number of proofs (expected 2, found 5)


False

In [17]:
tally = Tally()
vote = EncryptedVote()
vote.encrypted_answers = np.random.choice(encrypted_answers, 3) #EncryptedVote.fromElectionAndAnswers(election, answers)

tally.questions = {q: {"answers": range(num_choices)} for q in range(3)}
tally.public_key = keys.pk
tally.tally = [[0 for choice in range(num_choices)] for question in tally.questions]

tally.add_vote(vote, verify_p=False)
tally.add_vote(vote, verify_p=False)


result = decrypt_and_prove(tally, keys.sk)

# stops = [10**i for i in range(1, 7)]
# votes = []
# times = []

# for i in trange(1, stops[-1]+1):
#     tally.add_vote(vote, verify_p=False)
#     if i in stops:
#         election.encrypted_tally = tally
#         election.helios_trustee_decrypt()
#         trustees = Trustee.get_by_election(election)
#         decryption_factors = [t.decryption_factors for t in trustees]

#         tic = time.perf_counter()
#         result = tally.decrypt_from_factors(decryption_factors, election.public_key)

#         toc = time.perf_counter()
#         decrypt_time = toc - tic
#         print(f"#Votes: {i}, Tally-Time: {decrypt_time}")
#         votes.append(i)
#         times.append(decrypt_time)
#         # print(result)

In [18]:
result[0]

[[0, 0, 2, 0], [2, 0, 2, 0], [0, 2, 2, 2]]

In [19]:
vote.encrypted_answers[0].answer, vote.encrypted_answers[1].answer, vote.encrypted_answers[2].answer

([2], [0, 2], [1, 2, 3])

In [82]:
vote2[1].answer

[0, 1]